# MOVIE GENRE CLASSIFICATION 

# importing necessary libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm

# List of Genres learnt from the Training Data

In [2]:
genre_list = [ 'action', 'adult', 'adventure', 'animation', 'biography', 'comedy', 'crime', 'documentary', 'family', 'fantasy', 'game-show', 'history', 'horror', 'music', 'musical', 'mystery', 'news', 'reality-tv', 'romance', 'sci-fi', 'short', 'sport', 'talk-show', 'thriller', 'war', 'western' ]

# Define a fallback genre for movies which the model finds very hard to predict

In [3]:
fallback_genre = 'Unknown'

# Load the Training dataset from train_data.txt

In [5]:
try:
    with tqdm(total=50, desc="Loading Train Data") as pbar:
        train_data = pd.read_csv('C:/Users/deepa/Desktop/MOVIE GENRE CLASSIFICATION/train_data.txt', sep=':::', header=None, names=['SerialNumber', 'MOVIE_NAME', 'GENRE', 'MOVIE_PLOT'], engine='python')
        pbar.update(50)
except Exception as e:
    print(f"Error loading train_data: {e}")
    raise

Loading Train Data: 100%|█████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 207.94it/s]


# Data preprocessing for training data

In [6]:
X_train = train_data['MOVIE_PLOT'].astype(str).apply(lambda doc: doc.lower())
genre_labels = [genre.split(', ') for genre in train_data['GENRE']]
mlb = MultiLabelBinarizer()
y_train = mlb.fit_transform(genre_labels)

# TF-IDF Vectorization

In [7]:
tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # You can adjust max_features

# Fit and transform the training data with progress bar

In [8]:
with tqdm(total=50, desc="Vectorizing Training Data") as pbar:
    X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
    pbar.update(50)

Vectorizing Training Data: 100%|███████████████████████████████████████████████████████| 50/50 [00:03<00:00, 13.14it/s]


# Train a MultiOutput Naive Bayes classifier using the training data

In [9]:
with tqdm(total=50, desc="Training Model") as pbar:
    naive_bayes = MultinomialNB()
    multi_output_classifier = MultiOutputClassifier(naive_bayes)
    multi_output_classifier.fit(X_train_tfidf, y_train)
    pbar.update(50)

Training Model: 100%|█████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 126.67it/s]


# Load your test dataset from test_data.txt

In [10]:
try:
    with tqdm(total=50, desc="Loading Test Data") as pbar:
        test_data = pd.read_csv('C:/Users/deepa/Desktop/MOVIE GENRE CLASSIFICATION/test_data.txt', sep=':::', header=None, names=['SerialNumber', 'MOVIE_NAME', 'MOVIE_PLOT'], engine='python')
        pbar.update(50)
except Exception as e:
    print(f"Error loading test_data: {e}")
    raise

Loading Test Data: 100%|██████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 151.70it/s]


# Data preprocessing for test data

In [11]:
X_test = test_data['MOVIE_PLOT'].astype(str).apply(lambda doc: doc.lower())

# Transform the test data with progress bar

In [12]:
with tqdm(total=50, desc="Vectorizing Test Data") as pbar:
    X_test_tfidf = tfidf_vectorizer.transform(X_test)
    pbar.update(50)

Vectorizing Test Data: 100%|███████████████████████████████████████████████████████████| 50/50 [00:03<00:00, 13.30it/s]


# Predict genres on the test data

In [14]:
with tqdm(total=50, desc="Predicting on Test Data") as pbar:
    y_pred = multi_output_classifier.predict(X_test_tfidf)
    pbar.update(50)

Predicting on Test Data: 100%|████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 169.94it/s]


# Create a DataFrame for test data with movie names and predicted genres

In [15]:
test_movie_names = test_data['MOVIE_NAME']
predicted_genres = mlb.inverse_transform(y_pred)
test_results = pd.DataFrame({'MOVIE_NAME': test_movie_names, 'PREDICTED_GENRES': predicted_genres})

# Replace empty unpredicted genres with the fallback genre

In [16]:
test_results['PREDICTED_GENRES'] = test_results['PREDICTED_GENRES'].apply(lambda genres: [fallback_genre] if len(genres) == 0 else genres)
test_results['Serial_Number']=test_data['SerialNumber']
test_results['MOVIE_PLOT']=test_data['MOVIE_PLOT']

# Write the results to an output text file with proper formatting

In [17]:
with open("test_data_solution.txt", "w", encoding="utf-8") as output_file:
    for _, row in test_results.iterrows():
        id=row['Serial_Number']
        movie_name = row['MOVIE_NAME']
        genre_str = ', '.join(row['PREDICTED_GENRES'])
        plot=row['MOVIE_PLOT']
        output_file.write(f"{id} ::: {movie_name} ::: {genre_str} ::: {plot}\n")

# Calculate evaluation metrics using training labels (as a proxy)

In [18]:
y_train_pred = multi_output_classifier.predict(X_train_tfidf)

# Calculate evaluation metrics

In [19]:
accuracy = accuracy_score(y_train, y_train_pred)
precision = precision_score(y_train, y_train_pred, average='micro')
recall = recall_score(y_train, y_train_pred, average='micro')
f1 = f1_score(y_train, y_train_pred, average='micro')

# print accuracy precision recall f1-score

In [20]:
print("\n\nModel Evaluation Metrics:\n")
print(f"Accuracy: {accuracy * 100:.2f}%\n")
print(f"Precision: {precision:.2f}\n")
print(f"Recall: {recall:.2f}\n")
print(f"F1-score: {f1:.2f}\n")



Model Evaluation Metrics:

Accuracy: 27.73%

Precision: 0.72

Recall: 0.28

F1-score: 0.40

